In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
import shutil
import datetime
import matplotlib.pyplot as plt
import tabulate
from IPython.display import HTML, display, Markdown
import csv
import unicodedata
import getopt
from ipywidgets import HBox, VBox, Layout
from matplotlib.ticker import MaxNLocator

import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
import graphviz
from GraphVizUtil import *
from GraphVizFormats import *
import warnings
warnings.filterwarnings('ignore')
import seaborn as sb
sb.set(style='darkgrid', font_scale=1.2)

def filter_descriptions(tech_comm_desc):
    try:
        tech_comm_desc = tech_comm_desc.values[0][0].replace('#', '').replace('"','').replace("\n",'').strip()
    except:
        tech_comm_desc = 'No description provided'
    return tech_comm_desc

def create_args_flowd(df_graph):
    nodes, tech, ltech, to_tech, from_tech = set(), set(), set(), set(), set()
    for ind,row in df_graph.iterrows():
        #descriptions:
        input_comm_des = filter_descriptions(pd.read_sql("SELECT comm_desc FROM commodities WHERE comm_name='" + row['input_comm'] + "'", con))
        output_comm_des = filter_descriptions(pd.read_sql("SELECT comm_desc FROM commodities WHERE comm_name='" + row['output_comm'] + "'", con))
        tech_des = filter_descriptions(pd.read_sql("SELECT tech_desc FROM technologies WHERE tech='" + row['tech'] + "'", con))

        if 'ethos' in row['input_comm']:
            ltech.add('"' + row['tech'] + '"' +  ' [tooltip = "' + tech_des + '"]')
        else :
            nodes.add('"' + row['input_comm'] + '"' +  ' [tooltip = "' + input_comm_des + '"]')
        nodes.add('"' + row['output_comm'] + '"' +  ' [tooltip = "' + output_comm_des + '"]')
        tech.add('"' + row['tech'] + '"' +  ' [tooltip = "' + tech_des + '"]')

        if row['input_comm'] != 'ethos':
            to_tech.add('"%s"' % row['input_comm'] + '\t->\t"%s"' % row['tech']) 
        from_tech.add('"%s"' % row['tech'] + '\t->\t"%s"' % row['output_comm'])
    args = dict(
    enodes = "".join('%s;\n\t\t' % x for x in nodes),
    tnodes = "".join('%s;\n\t\t' % x for x in tech),
    iedges = "".join('%s;\n\t\t' % x for x in to_tech),
    oedges = "".join('%s;\n\t\t' % x for x in from_tech),
    snodes = ";".join('%s' %x for x in ltech),
    )
    return args


def return_format_colors():
    colors = {}
    colors.update(getColorConfig(False))
    return colors, quick_run_dot_fmt

def return_flowd_table(final_dem, level=1):
    df = pd.read_sql("SELECT * FROM Efficiency", con)
    df_sel = df[df['output_comm']==final_dem]
    if len(df_sel)==0:
        df_sel = df[df['tech']==final_dem]
    inputs = df_sel['input_comm'].unique()
    iterval=0
    if level!=0:
        while len(inputs)>0:
            df_append = df[df['output_comm'].isin(inputs)]
            df_sel = pd.concat([df_sel, df_append])
            inputs = df_append['input_comm'].unique()
            iterval+=1
            if iterval>level-1:
                break
    df_graph = df_sel[['input_comm', 'tech', 'output_comm']].drop_duplicates()
    return df_graph

def return_flowd_table_fwds(final_dem):
    df = pd.read_sql("SELECT * FROM Efficiency", con)
    df_sel = df[df['output_comm']==final_dem]
    if len(df_sel)==0:
        df_sel = df[df['tech']==final_dem]
    inputs = df_sel['input_comm'].unique()
    outputs = df_sel['output_comm'].unique()

    iterval=0
    while len(inputs)>0:
        df_append = df[df['output_comm'].isin(inputs)]
        df_sel = pd.concat([df_sel, df_append])
        inputs = df_append['input_comm'].unique()
        iterval+=1
        if iterval>2:
            break
    iterval=0
    while len(outputs)>0:
        df_append = df[df['input_comm'].isin(outputs)]
        df_sel = pd.concat([df_sel, df_append])
        outputs = df_append['output_comm'].unique()
        iterval+=1
        if iterval>=0:
            break

    df_graph = df_sel[['input_comm', 'tech', 'output_comm']].drop_duplicates()
    return df_graph

con = sqlite3.connect(r'../US_9R_4D.sqlite') #change path to database
cur = con.cursor()   
con.text_factory = str 

def controls_rows(w):
    controls = HBox(w.children[:-1], layout = Layout(flex_flow='row wrap', width='max-content'))
    output = w.children[-1]
    display(VBox([controls, output],  layout = Layout(flex_flow='columns wrap', width='max-content', size=10)))
    

# Buildings Sector Overview <a class="anchor" id="buildings"></a>
In the OEO database, the buildings sector is divided into the residential and commercial sectors. This notebook provides detail on two key components that describe these sectors: 1) end-use service demands and 2) the demand technologies used to meet those demands. The service demands are derived from the <a href="https://www.nrel.gov/analysis/electrification-futures.html" rel="nofollow">NREL Electrification Futures Study</a> and consist of two components: 1) the region-specific annual demands projected to 2050 and 2) the apportionment of these annual demands at the hourly level. The demand technologies are characterized by their capital costs, fixed and variable operation and maintenance costs, efficiencies, capacity factors, existing capacity, and fuel inputs.

## Service Demands <a class="anchor" id="service_demands"></a>
The <a href="https://www.nrel.gov/analysis/electrification-futures.html" rel="nofollow">NREL Electrification Futures Study</a> used the <a href="https://www.evolved.energy/post/2016/02/19/energypathways" rel="nofollow">EnergyPATHWAYS</a> tool to develop a set of energy demand and supply projections for
the end-use sectors in the US out to 2050. These annual projections are available at the state level for subsectors such as residential/commercial space heating, air conditioning, cooking, and water heating. The study also has state-level projections of hourly electricity demands for some of these subsectors. We are currently using these two datasets to develop demand projections for the OEO database. In the future, we plan to utilize more detailed projections based on <a href="https://www.nrel.gov/buildings/end-use-load-profiles.html" rel="nofollow">this multi-year NREL study</a> that is currently in progress.


### Annual Demand Projections <a class="anchor" id="annual_demands"></a>
The <a href="https://www.nrel.gov/docs/fy18osti/71500.pdf" rel="nofollow">NREL Electrification Futures Study (EFS) report</a> published in 2018 presents a range of demand scenarios for all energy sectors in the United States. We used the estimates of buildings-related service demands for the end-use demands listed in Table 1 available for download <a href="https://data.nrel.gov/submissions/92" rel="nofollow">here</a>. EFS uses EIA service demand data at the US census division-level and downscales this to the state-level using end-use demand specific factors. For example, residential space heating service demands were downscaled using the product of each US state's share of heating-degree-days and residential square footage. In our databases we perform an additional aggregation to represent the state-level demands from EFS at the OEO region-level (shown in Figure 1). Commercial ventilation is an exception in that the the annual service demands were estimated from <a href="https://www.eia.gov/outlooks/aeo/data/browser/#/?id=5-AEO2021&cases=ref2021&sourcekey=0" rel="nofollow">AEO</a> as the product of the commercial square footage of a region (collection of US states) times a ventilation efficiency times the ventialtion energy consumption. This was done to resolve inconsistencies across other parameters (specfically existing capacity and technology efficiencies) across EFS and AEO datasets - in part because EFS reports service demands for commercial ventialtion in giga-cubic-feet as opposed to the AEO estimate which is in peta-Joules.


**Table 1.** Annual service demand categories and units drawn from the NREL Electrification Futures Study (unless noted otherwise) currently used in the OEO buildings database. The demand commodity names used in the OEO input database are also tabulated.

| End-Use Demand | Unit | OEO Demand Commodity |
|:-|:-|:-|
| Commercial Space Cooling <img width=15/> | TERA-BTU |CSC<img width=15/>|
| Commercial Cooking | TERA-BTU | CCK |
| Commercial Lighting | GIGA-LUMEN-YEAR |CLT |
| Commercial   Refrigeration | TERA-BTU | CRF |
| Commercial Space   Heating | TERA-BTU | CSH |
| Commercial   Ventilation <sup>[1](#AEO_DATA)</sup>| TRILLION-CUBIC-FEET-PER-MINUTE-HOUR | CVT |
| Commercial Water   Heating | TERA-BTU | CWH |
| Commercial   Other gas <sup>[2](#COMM_OTHER)</sup>| MMBTU | COTHER_NGA |
| Commercial   Other electricity <sup>[2](#COMM_OTHER)</sup>| MMBTU | COTHER_ELEC |
| Commercial   Other diesel <sup>[2](#COMM_OTHER)</sup>| MMBTU | COTHER_DSL |
| Residential   Space Cooling | MMBTU | RSC |
| Residential Refrigeration | MEGA-CUBIC-FEET | RRF |
| Residential   Lighting | GIGA-LUMEN-YEAR | RLT |
| Residential   Space Heating | MMBTU | RSH |
| Residential   Water Heating | MMBTU | RWH |
| Residential   Clothes Drying | GIGA-LB | RCD |
| Residential   Dishwashing | GIGA-CYCLE | RDW |
| Residential   Freezing | MEGA-CUBIC-FEET | RFZ |
| Residential   Cooking | MMBTU | RCK |
| Residential   Other electricity <sup>[3](#RES_OTHER)</sup>| MMBTU | ROTHER_ELEC |

<a name="AEO_DATA">1</a>: Service demand estimated from AEO as commercial regional square footage $\times$ ventilation efficiency $\times$ ventilation energy consumption
<a name="COMM_OTHER">2</a>: The commercial other category in the OEO databases aggregates the service demands reported by EFS for the categories: 'Commercial Other', 'District Services', 'Office Equipment (NON-P.C.)' and 'Office Equipment (P.C.)'.
<a name="RES_OTHER">3</a>: The residential other category in the OEO databases aggregates the service demands reported by EFS for the categories: 'Televisions and related', 'Computers and related', 'residential other uses' and 'residential furnace fans'.

In [2]:
def show_demand(con):

    display_types = ['table', 'figure']

    df_demand = pd.read_sql("SELECT * FROM Demand WHERE demand_comm in (SELECT comm_name FROM commodities WHERE flag='d') \
                            AND demand_comm in (SELECT output_comm FROM Efficiency WHERE tech IN (SELECT tech FROM technologies WHERE sector='residential' OR sector='commercial'))", con)
   
    df_tech_desc = pd.read_sql("SELECT comm_name, comm_desc FROM commodities", con)
    df_demand['demand_comm_desc'] = df_demand['demand_comm'].map(lambda x: df_tech_desc.loc[df_tech_desc.comm_name==x,'comm_desc'].values[0].replace('#','').strip())

    demand_comms = []
    for unique_demand_comm in df_demand.demand_comm.unique():
        demand_comms.append(unique_demand_comm)
    demand_comms = list(set(demand_comms))
    regions = df_demand.regions.unique()


    def filter_demand_comm(demand_comm='', o_format=''):
        demand_comm_name = pd.read_sql("SELECT comm_desc FROM commodities where comm_name=='" + demand_comm +"'", con)
        df_sel = df_demand[(df_demand.demand_comm==demand_comm)]
        df_sel = df_sel.pivot_table(
            index=['regions', 'demand_comm', 'demand_units', 'demand_comm_desc'],
            columns='periods',
            values='demand').reset_index().set_index('regions')
        #df_sel['demand_comm'] = demand_comm_name.iloc[0,0].replace('#','').strip()
        df_sel.rename(columns={'demand_units': 'units', 'demand_comm':'demand', 'demand_comm_desc':'demand description'}, inplace=True)
        df_sel.units = df_sel.units.str.replace('#','').str.strip()
        if o_format == 'table':
            display(
                HTML(
                    tabulate.tabulate(df_sel,
                                      ['region'] + list(df_sel.columns.values),
                                      floatfmt=".1f",
                                      tablefmt='html')))
        elif o_format == 'figure':
            fig, ax = plt.subplots(figsize=(10, 6))
            df_sel.drop(columns=['demand', 'units', 'demand description']).transpose().plot.bar(stacked=True, ax = ax)
            plt.legend(bbox_to_anchor=(0, 1), loc='upper left', ncol=1)
            handles, labels = ax.get_legend_handles_labels()
            ax.legend(handles[::-1], labels[::-1], loc='upper right',bbox_to_anchor=(1.2, 1))
            plt.ylabel(demand_comm_name.iloc[0,0].replace('#','').strip() + ' (' + df_sel.units.unique()[0].replace('#','') + ')')
            plt.xlabel('')
    w2 = widgets.Select(options=demand_comms, description='demand')
    w3 = widgets.Select(options=display_types)

    w_filter = widgets.interactive(filter_demand_comm, demand_comm=w2, o_format=w3)
    controls_rows(w_filter)

display(Markdown('Annual end-use OEO regional demand projections to 2050 can be explored using the tool below'))
show_demand(con)

Annual end-use OEO regional demand projections to 2050 can be explored using the tool below

###  Hourly Demand Representation <a class="anchor" id="hourly_demands"></a>

We use the [electricity demand profiles](https://data.nrel.gov/submissions/126) published by NREL for the demand categories listed in Table 2 to approximate service demand patterns. Here we assume that the service demands follow the same pattern as hourly electricity demand. These profiles were developed by NREL using the outputs of other models such as ResStock/ComStock and other data (e.g., from metering studies) for a range of future scenarios. The results from the 'High-Electrification and Rapid End-use Technology Advancement' scenario are selected as the electrification profiles in our database as it incorporates the largest share of total service demands. In cases where the hourly profiles of a sub-sector are not available, we assume service demands are constant throughout the year. 

**Table 2.** Hourly service demand categories as reported by NREL EFS for the residential and commercial sectors in the OEO database. 

| Sector | Sub-sector |
|:-|:-|
| commercial <img width=15/> |space heating and cooling<img width=15/>|
| commercial | water heating |
| commercial | other |
| residential | space heating and cooling |
| residential | water heating |
| residential | clothes drying/washing |
| residential | other |

<img src="documentation_images/IPM_regions_to_temoa_state_groups.jpg" width="60%"/>

**Figure 1.** The nine US regions developed for the electric sector in the OEO input database, based on aggregations of IPM regions

The raw data from EFS combines space heating and cooling hourly load profiles. To separate the heating and cooling profiles we use population weighted heating and cooling degree hour (hdh/cdh) data provided by Michael Roberts at the University of Hawaii (described in a forthcoming paper). This data is available at the hourly timescale for the year 2010 and is available across the IPM regions. We first calculate an average hdh for IPM regions within a given OEO region (Figure 1) for each hour of 2010. Next, we calculate the hourly hdh fraction as: 
\begin{equation*}
\frac{\bar{hdh}}{\bar{hdh} + \bar{cdh}}
\end{equation*}
where $\bar{hdh}$ and $\bar{cdh}$ are the hourly hdh and cdh, respectively, averaged across IPM regions in a given OEO region. These fractions are calculated for each hour in each OEO region. Finally, the combined heating and cooling load from NREL EFS is multiplied by these fractions to estimate the heating load in each hour. The cooling load is similarly estimated as the total load multiplied by the fraction of cooling degree hours over the sum of heating and cooling degree hours. This disaggregation of hourly heating and cooling load is illustrated in Figure 2 for two OEO regions.

![Figure 2: hdh and hdh fraction for OEO regions](./documentation_images/hdh_cdh_OEO.png)
**Figure 2.** Heating degree hour (hdh) and fraction of heating degree hour for the OEO ‘TX’ (left) and 'CEN' (right) region. The markers on the plot show the hourly hdh for the IPM regions within the given OEO region. The line shows the average hdh fraction for each hour smoothed with a moving average window of 20 hours for legibility.

In [3]:
display(Markdown('The hourly demand profiles for residential/commercial end use demand are shown below.'))

def show_hourly_dem(conn):
    df_dds = pd.read_sql_query('SELECT * FROM DemandSpecificDistribution', conn)
    regions = df_dds.regions.unique()
    unique_demands = df_dds.demand_name.unique()
    
    def select_region(region='', dem_name=''):
        df_sel = df_dds.loc[(df_dds.regions==region) & (df_dds.demand_name==dem_name)].copy()
        fig, ax = plt.subplots(figsize=(10,6))
        plt.plot(np.arange(0, len(df_sel['dds'])),df_sel['dds'], label=dem_name)
        plt.ylabel('Hourly demand fraction')
        plt.xlabel('Hour')
        plt.legend(loc='upper right',bbox_to_anchor=(1.2, 1))

    w1 = widgets.Select(options=regions)
    w2 = widgets.Select(options=unique_demands)
    w = widgets.interactive(select_region, region=w1, dem_name=w2)
    
    controls_rows(w)

show_hourly_dem(con)

The hourly demand profiles for residential/commercial end use demand are shown below.

## Demand Technology Specification <a class="anchor" id="demand_technology"></a>

The characteristics of the demand technologies in the residential and commercial sectors in the OEO database are based on the Residential Demand Module (RDM) and Commercial Demand Module (CDM) of the National Energy Modeling System (NEMS) - [Updated Buildings Sector Appliance and Equipment Costs and Efficiency](https://www.eia.gov/analysis/studies/buildings/equipcosts/). In our database we incorporate the **1) technology-specific efficiencies**, **2) fixed and variable operations and maintenance costs**, **3) investment costs**, **4) lifetimes**, and 5) **typical capacities** reported in the appendices which are based on contract reports prepared by Navigant Consulting, Inc. for the U.S. Energy Information Administration. In this way we are able to represent a diverse set of equipment classes/types capable of servicing different parts of the buildings sector. For example, the technologies capable of meeting residential cooling demand include room air conditioners, central air conditioners, and heat pumps. Three types of heat pumps are represented for residential cooling: air-source, ground-source, and natural gas heat pumps. We used this dataset as-is, except for an adjustment of the heat pump coefficient of performance (COP) to reflect more regionally appropriate values. [Vaishnav et al. (2020)](https://pubs.acs.org/doi/abs/10.1021/acs.est.0c02705) used a dataset gathered from NEEP and estimated a linear relationship between COP and outside temperature for a variety of heat pumps. Their approach assumed that the indoor temperatures stay constant at 70 F. We use the slope of this linear relationship to adjust heat pump COPs at the OEO region level using the NEMS database and applying state-level population weighted temperatures.

Along with diversity in technology representations, the appendices above project the techno-economic parameters for residential and commercial equipment from 2020 to 2050 in 5 year increments. In most cases, two versions of the same technology for a given vintage are included in our database: 1) a standard version and 2) a high efficiency version. For example: The heating seasonal performance factors (HSPF) for a typical air-source heat pump of a 2020 vintage is 8.6 whereas a high-efficiency version has an HSPF of 9.0 (the costs, lifetimes may also vary across these different versions). These different versions of technologies are utilized as part of the OEO buildings database. The techno-economic parameters are assumed to be region agnostic with the exception of heat pump COPs as described above. 

Input values drawn directly from the database can be viewed in the sections below.

### Investment costs <a class="anchor" id="costinvest"></a>

The investment costs are drawn from the [Residential Demand Module (RDM) and Commercial Demand Module (CDM)](https://www.eia.gov/analysis/studies/buildings/equipcosts/) of National Energy Modeling System. Specific values can be inspected using the look up tool below.

In [4]:
def show_cost_invest(con):
    
    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)

    df = pd.read_sql(
        "SELECT regions, tech,vintage, cost_invest, cost_invest_units FROM CostInvest WHERE tech IN (SELECT tech FROM technologies WHERE sector=='residential' OR sector='commercial' ) ORDER BY tech, vintage",
        con)

    display_types = ['table', 'figure']
    
    df['agg_tech'] = df['tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())
    techs = []
    for unique_tech in df.agg_tech.unique():
        try:
            int(unique_tech.split('_')[-1])
            techs.append('_'.join(unique_tech.split('_')[0:-1]))
        except:
            techs.append(unique_tech)
    techs = ['All'] + list(set(techs))
    regions = df.regions.unique()


    def filter_tech(tech='', region='', o_format=''):
        if tech == 'All':
            df_sel = df[(df.regions == region)]
        else:
            df_sel = df[(df.agg_tech.str.contains(tech, regex=False)) & (df.regions == region)]
        if o_format == 'table':
            df_sel = df_sel.pivot_table(
                index=['regions', 'tech', 'agg_tech', 'cost_invest_units'],
                columns='vintage',
                values='cost_invest').reset_index().set_index('regions')
            df_sel.rename(columns={'cost_invest_units': 'units', 'agg_tech':'description'}, inplace=True)
            df_sel['units'] = df_sel['units'].str.replace('#','').str.replace('M$','$M').str.strip()
            if len(df_sel['units'].unique())==1:
                df_sel['units'] = df_sel['units'].unique()[0].replace('M$','$M')
            display(
                HTML(
                    tabulate.tabulate(df_sel,
                                      ['region'] + list(df_sel.columns.values),
                                      floatfmt=".0f",
                                      tablefmt='html')))
        elif (o_format == 'figure') & (tech!='All') :
            fig, ax = plt.subplots(figsize=(10, 6))
            for ind_tech in df_sel.tech.unique():
                plt.plot(df_sel[df_sel.tech == ind_tech].vintage,
                         df_sel[df_sel.tech == ind_tech].cost_invest,
                         label=ind_tech)
            plt.legend()
            plt.ylabel('Investment costs (' + df_sel['cost_invest_units'].unique()[0].replace('#','').strip().replace('M$','$M') + ')')
            #plt.ylim([0, df.cost_invest.max() * 1.1])
            plt.xlabel('Vintage')


    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w3 = widgets.Select(options=display_types)
    w = widgets.interactive(filter_tech, tech=w1, region=w2, o_format=w3)

    controls_rows(w)
    
    
    
show_cost_invest(con)

### Efficiency <a class="anchor" id="efficiency"></a>

Our representation of the buildings sector is comprised of several end-use services like residential space heating and commercial ventilation (listed in Table 1). These end-use service demands are met by a host of demand technologies that consume energy commodities and produce end-use demand commodities. In this section, we consider the numerous efficiency metrics that allow for this conversion and how they are represented in the OEO buildings database.

Table 3 summarizes the end-use service demand, the efficiency metrics of the technologies used to service that demand and the units of the efficiency metric in the OEO database. The demands for some end-use services (like residential space heating) are specified in energy units and so the efficiencies of technologies meeting these demands are in units of energy output over energy input supplied. However, other end-use services (like commercial ventilation) have demand specified in physical units, and thus the efficiencies of the associated demand technologies are in physical units produced over input energy supplied. 

**Table 3.** Summary of technology efficiencies applied to building sub-sectors in the OEO database. 

| End-Use Demand | Efficiency Metrics | Efficiency Units
|:-|:-|:-|
| Commercial Space Cooling <img width=15/> | COP<sup>[1](#COP)</sup>, EER<sup>[2](#EER)</sup>, IEER<sup>[2](#EER)</sup> <img width=15/>| PJ-out/PJ-In<img width=15/>|
| Commercial Cooking | cooking energy efficiency | PJ-out/PJ-In |
| Commercial Lighting | system efficacy<sup>[3](#LIGHT)</sup> | Giga-Lumen-Year/PJ |
| Commercial Refrigeration | Nominal Capacity Over Average Input | PJ-out/PJ-In |
| Commercial Space Heating | COP, thermal efficiency<sup>[4](#TE)</sup>, COP, AFUE<sup>[5](#AFUE)</sup>| PJ-out/PJ-In |
| Commercial Ventilation | ventilation efficiency | Trillion-CFM-hour/PJ-in |
| Commercial Water Heating | thermal efficiency, SEF<sup>[7](#SEF)</sup>, COP | PJ-out/PJ-In |
| Residential Space Cooling | SEER<sup>[6](#AFUE)</sup>, EER, COP | PJ-out/PJ-In |
| Residential Refrigeration | typical capacity over annual energy consumption | Mega-Cubic-Feet/PJ |
| Residential Lighting | system efficacy | Giga-Lumen-Year/PJ |
| Residential Space Heating | AFUE, thermal efficiency, HSPF<sup>[8](#HSPF)</sup>, COP, HHV<sup>[9](#HHV)</sup>| PJ-out/PJ-In |
| Residential Water Heating | UEF<sup>[7](#SEF)</sup>, SEF<sup>[7](#SEF)</sup> | PJ-out/PJ-In |
| Residential Clothes Drying | Combined Energy Factor | Giga-Lb/PJ |
| Residential Dishwashing | cycles per year over annual energy use | Giga-Cycles/PJ |
| Residential Freezing | typical capacity over annual energy consumption | Mega-Cubic-Feet/PJ |
| Residential Cooking | cooking energy efficiency | PJ-out/PJ-in |

<a name="COP">1</a>: Coefficient of Performance (COP):  Energy efficiency rating measure determined, under specific testing conditions, by dividing the useful heating or cooling delivered by the required energy input.
<a name="EER">2</a>: Energy Efficiency Ratio (EER, IEER): A ratio representing the cooling capacity in Btu per hour by the power input in watts at any given set of rating conditions, expressed in Btu per hour per watt.
<a name="LIGHT">3</a>: Lighting efficiencies are represented by efficacy in lumens/watt.
<a name="TE">4</a>: Thermal efficiency (TE): the percentage of input heat energy that is transformed into useful work.
<a name="AFUE">5</a>: Annual Fuel Utilization Efficiency (AFUE): Efficiency rating based on average usage, including on and off cycling, as set out in the Department of Energy's standardized test procedures.
<a name="SEER">6</a>: Seasonal Energy Efficiency Ratio (SEER): The total cooling of a central unitary air conditioner or a unitary heat pump in Btu during its normal annual usage period for cooling divided by the total electric energy input in watt-hours during the same period. 
<a name="SEF">7</a>: Solar Energy Factor (SEF) and Uniform Energy Factor (UEF): defined as the energy delivered by the system divided the electrical or gas energy put into the system.
<a name="HSPF">8</a>: Heating Seasonal Performance Factor (HSPF): The total heating delivered by a heat pump in Btu during its normal annual usage period for heating divided by total electric input in watt-hours during the same period.
<a name="HHV">9</a>: Higher Heating Value (HHV): This thermal efficiency is fuel dependent and accounts for the latent heat of vaporization of water in the combustion products.

As noted above, several end-use service demands are defined by units of service rather than energy. In these cases, the specified efficiency has different input and output units:

- **Commercial Lighting**:
Here conversion from electrical energy to lumens is represented by the system efficacy. The system efficacy is published in $\frac{Lumens}{Watt}$ and converted to $\frac{Giga-Lumens\cdot year}{PJ}$ by multiplying by $ 10^{-9} \frac{Giga}{1} / (3600 \frac{J}{Wh} \times 10^{-15} \frac{PJ}{J} \times 8760 \frac{hours}{year}) $.
- **Commercial Ventilation**:
These conversion factors are taken from the Commercial Demand Module of the National Energy Modeling System as $\frac{Trillion-CFM-hours}{PJ}$
- **Residential Refrigeration**:
An efficiency is derived as the typical capacity over annual energy consumption where typical capacity is in units of ft<sup>3</sup> and annual energy consumption is in the units of kWh and subsequently converted to $\frac{ft^3}{PJ}$ by dividing by $ 3.6\times 10^{-9} \frac{PJ}{kWh} $.
- **Residential Lighting**:
Here conversion from electrical energy to lumens is represented by the system efficacy. The system efficacy is published in $\frac{Lumens}{Watt}$ and converted to $\frac{Giga-Lumens\cdot year}{PJ}$ by multiplying by $ 10^{-9} \frac{Giga}{1} / (3600 \frac{J}{Wh} \times 10^{-15} \frac{PJ}{J} \times 8760 \frac{hours}{year}) $.
- **Residential Clothes Drying**:
The combined energy factor in the [EIA appendicies](https://www.eia.gov/analysis/studies/buildings/equipcosts/) are presented in units of $\frac{lb}{kWh}$ and converted to $\frac{Giga-lb}{PJ}$ by multiplying by $10^{-9}\frac{Giga}{1} / (3.6\times10^{-9}\frac{PJ}{kWh})$
- **Residential Dishwashing**:
An efficiency is derived as the number of cycles in year over the typical annual energy use. An average of 215 cycles/year is assumed as per the [EIA appendicies](https://www.eia.gov/analysis/studies/buildings/equipcosts/). The calculated efficiency is presented in units of $\frac{cycles}{kWh}$ and converted to $\frac{Giga-cycles}{PJ}$ by multiplying by $10^{-9}\frac{Giga}{1} / (3.6\times10^{-9}\frac{PJ}{kWh})$
- **Residential Freezing**:
An efficiency is derived as the typical capacity over annual energy consumption where typical capacity is in units of ft<sup>3</sup> and annual energy consumption is in the units of kWh and subsequently converted to $\frac{mega ft^3}{PJ}$ by multiplying by $ \frac{10^{-6}}{3.6\times 10^{-9}} \frac{PJ}{kWh} $.


To calculate efficiencies for commercial refrigeration technologies, the following approach was adopted:
- **Commercial Refrigeration**:
An efficiency is derived as the nominal capacity over average input, calculated as the cooling or heat rejection capacity over the annual energy consumption with units of $\frac{PJ in}{PJ out}$, where the cooling or heat rejection capacity is in units of $\frac{Btu}{hour}$ and the annual energy consumption is in units of $\frac{kWh}{Year}$. The unit conversions are as follows (Note that $\frac{PJ out}{PJ in} = \frac{Btu out}{But in}$ since the conversion factors cancel out):
\begin{equation*}
\frac{Capacity \frac{Btu}{hour} \times 24 \frac{hours}{day} \times 365 \frac{day}{year}}{Annual Energy Consumption \frac{kWh}{Year} \times 3421.14 \frac {Btu}{kWh}}
\end{equation*}.

In [5]:
def show_efficiency(con):
    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)
    df = pd.read_sql("SELECT regions, tech, vintage, efficiency, eff_notes FROM Efficiency WHERE tech IN (SELECT tech FROM technologies WHERE sector=='residential' OR sector='commercial')", con)
    df.loc[:,'agg_tech'] = df.loc[:,'tech'] #[map_plants[y] for x in df.tech for y in map_plants.keys() if y.lower() in x.lower()] #map agg technologies
    df_sum = df#df.drop("vintage", axis=1).groupby(by = ['regions','agg_tech']).sum().reset_index()
    df_sum['agg_tech'] = df_sum['agg_tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())
    techs = ['All'] + list(df_sum.agg_tech.unique())
    regions = df_sum.regions.unique()
    def filter_tech(tech ='', region = ''):
        if tech=='All':
            df_sel = df_sum[(df_sum.regions==region)]
        else:
            df_sel = df_sum[(df_sum.agg_tech==tech) & (df_sum.regions==region)]
#             df_sel.efficiency = 100*df_sel.efficiency
        df_sel = df_sel[['regions','tech','agg_tech','vintage','efficiency', 'eff_notes']]
        display(HTML(tabulate.tabulate(df_sel.set_index('regions'), ["regions", "technology", "description","vintage",'efficiency', 'efficiency units' ], floatfmt=".1f", tablefmt='html')))

    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w = widgets.interactive(filter_tech, tech=w1, region=w2)

    controls_rows(w)
    
show_efficiency(con)

### Existing Capacity <a class="anchor" id="exist_cap"></a>

To our knowledge, the existing installed capacities of the various technologies listed in the [EIA dataset](https://www.eia.gov/analysis/studies/buildings/equipcosts/) are not readily available. Here we rely on two main data source to estimate existing capacity at the technology level for each sub-sector in the OEO buildings database: **1) NREL EFS service demands scaled using a derived utilization factor** and **2) USEPA9r MARKAL database**. Table 3 provides a summary of which data sources are used for each of the represented sub-sectors.

**Table 3.** Summary of data sources used to estimate existing installed capacity for the buildings sub-sectors in the OEO database. 

| End-Use Demand | Data Source |
|:-|:-|
| Commercial Space Cooling <img width=15/> | NREL EFS<sup>[1](#EFS)</sup> <img width=15/>| 
| Commercial Cooking | NREL EFS | 
| Commercial Lighting | EPA MARKAL<sup>[2](#MARKAL)</sup> | 
| Commercial Refrigeration | NREL EFS | 
| Commercial Space Heating | NREL EFS | 
| Commercial Ventilation | EPA MARKAL | 
| Commercial Water Heating | NREL EFS | 
| Residential Space Cooling | NREL EFS | 
| Residential Refrigeration | NREL EFS | 
| Residential Lighting | EPA MARKAL | 
| Residential Space Heating | NREL EFS | 
| Residential Water Heating | EPA MARKAL | 
| Residential Clothes Drying | NREL EFS | 
| Residential Dishwashing | NREL EFS | 
| Residential Freezing | NREL EFS | 
| Residential Cooking | NREL EFS |

<a name="EFS">1</a>: [NREL Electrification Futures Study (EFS)](https://www.nrel.gov/docs/fy18osti/71500.pdf).
<a name="MARKAL">2</a>: [USEPA9r MARKAL database (Shay et al. 2013)](https://cfpub.epa.gov/si/si_public_record_Report.cfm?Lab=NRMRL&dirEntryID=150883)

The estimation method from the two data sources are briefly described below:

**1) NREL Electrification Futures Study**: First an end-use service demand specific 'utilization factor' is calculated using the hourly load profile data published in the EFS. This is done by calculating the average demand across the 8760 hours divided by the 95th percentile demand for each end-use service demand in each OEO region for which hourly load profiles are available. The existing capacity of the builings sector technologies is then estimated as the service demand in 2017 from EFS scaled using the calculated utilization factors.

**2) USEPA9r MARKAL database**: Here we lean on existing capacity estimations from the USEPA9r MARKAL database [(Shay et al. 2013)](https://cfpub.epa.gov/si/si_public_record_Report.cfm?Lab=NRMRL&dirEntryID=150883). MARKAL spreadsheets report the existing capacities for most of the technologies listed in the EIA dataset for the [nine Census Divisions](https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf). However, since the OEO regions differ from the Census Divisions, we scale the reported existing capacities using [U.S. Census state population data](https://www.census.gov/data/tables/time-series/demo/popest/2010s-state-total.html) to obtain estimates of existing capacity in each U.S. state. Finally, we aggregate up to the OEO regions and estimate the existing capacity of the EIA technologies. 

MARKAL estimates the existing capacity by multiplying the demand met (taken from the [AEO](https://www.eia.gov/outlooks/aeo/)) in an end-use service sub-sector by the estimated market share of a technology contributing to the end-use service category. This value is then divided by a utilization factor to estimate existing capacity of a certain technology in a given region. We directly use the calculated existing capacities in the OEO databases for the end-use demands as listed in Table 3.

In [6]:
def show_exist_cap(con):
    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)
    df = pd.read_sql("SELECT regions, tech,vintage, exist_cap, exist_cap_units FROM ExistingCapacity WHERE tech IN (SELECT tech FROM technologies WHERE sector=='residential' OR sector='commercial')", con)
    df.loc[:,'agg_tech'] = df.loc[:,'tech'] #[map_plants[y] for x in df.tech for y in map_plants.keys() if y.lower() in x.lower()] #map agg technologies
    df_sum = df.drop("vintage", axis=1).groupby(by = ['regions','tech','agg_tech','exist_cap_units']).sum().reset_index()
    df_sum.sort_values(by='exist_cap', ascending=False, inplace=True)
    df_sum['agg_tech'] = df_sum['agg_tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())

    df_sum[['exist_cap_units']] = 'to be updated' # this line needs to be deleted once the units for existing capacity have been updated directly in the database
    
    techs = ['All'] + list(df_sum.agg_tech.unique())
    regions = df_sum.regions.unique()
    def filter_tech(tech ='', region = ''):
        if tech=='All':
            df_sel = df_sum[(df_sum.regions==region)]
        else:
            df_sel = df_sum[(df_sum.agg_tech==tech) & (df_sum.regions==region)]
        display(HTML(tabulate.tabulate(df_sel.set_index('regions'), ["regions", "technology", "description", "units","capacity" ], floatfmt=".1f", tablefmt='html')))
    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w = widgets.interactive(filter_tech, tech=w1, region=w2)

    controls_rows(w)
    
show_exist_cap(con)


### Discount Rate <a class="anchor" id="disc_rates"></a>
All new capacity is given a technology-specific discount rate of 30%. Without this specification, TEMOA's least cost optimization results in the early replacement of virtually all existing demand technologies with new capacity at higher efficiency. This 30% discount rate is used to ensure that existing capacity is utilized for the remainder of its lifetime under base case assumptions. In the future, we plan to update this hurdle rate based on calibrations from the updated OEO database.

In [7]:
def show_disc_rate(con):
    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)
    df = pd.read_sql("SELECT regions, tech, vintage, tech_rate FROM DiscountRate \
    WHERE tech in (SELECT tech FROM technologies WHERE sector='residential' OR sector='commercial')", con)
    df['agg_tech'] = df['tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())
    techs = []
    for unique_tech in df.agg_tech.unique():
        techs.append(unique_tech)
    techs = ['All'] + list(set(techs))
    regions = df.regions.unique()
    def filter_tech(tech='', region =''):
        if tech=='All':
            df_sel = df[(df.regions==region)]
        else:
            df_sel = df[(df.agg_tech==tech) & (df.regions==region)]
        df_sel = df_sel.pivot_table(index=['regions','tech', 'agg_tech'], columns='vintage', values='tech_rate').reset_index()
        df_sel.rename(columns={'agg_tech':'description'},inplace=True)
        header = list(df_sel.columns.values)
        display(HTML(tabulate.tabulate(df_sel.set_index('regions'), header, floatfmt=".3f", tablefmt='html')))
    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w = widgets.interactive(filter_tech, tech=w1, region=w2)

    controls_rows(w)
    
show_disc_rate(con)

### Tech Input Split <a class="anchor" id="tech_input_split"></a>

We specify minimum shares of commodity inputs to residential space heating, commerical space heating, commercial ventilation and commercial refrigeration in the 2020 time period to match historical demand. The shares are determined using 2020 service demand data from NREL EFS. These constraints are lifted for all future time periods in the model allowing it to make decisions on technology adoption.

## Look-up Tools <a class="anchor" id="look_up_tools"></a>

### Technology/Commodity Description Look-up Tool <a class="anchor" id="description_look_up"></a>
Use the tool below to search for any key words that may describe a technology or commodity of interest (e.g. heating, cooling). The tool provides a list of all the technologies in the database that may be relevant to the query.

In [8]:
w = widgets.Text(value='space heating demand')
display(w)
def f(w):
    if len(w)>0:
        df1 = pd.read_sql("SELECT * FROM commodities WHERE comm_desc LIKE '%" + w + "%'", con)
        df1['desc'] = df1['comm_desc'].str.replace('#','').str.strip()
        df1['comm_tech'] = df1['comm_name']
        df1['type'] = 'commodity'

        df2 = pd.read_sql("SELECT * FROM technologies WHERE tech_desc LIKE '%" + w + "%'", con)
        df2['desc'] = df2['tech_desc'].str.replace('#','').str.strip()
        df2['comm_tech'] = df2['tech']
        df2['type'] = 'technology'


        df = pd.concat([df1[['comm_tech','type','desc']], df2[['comm_tech','type','desc']]])
        
        if len(df)>0:
            display(HTML(tabulate.tabulate(df.set_index('comm_tech'),['technology/commodity','type','description'],tablefmt='html')))
        else:
            print('')
    else:
        print('')
            


out = widgets.interactive_output(f, {'w': w})
display(out)

Text(value='space heating demand')

Output()

### Network Diagram Look-up Tool  <a class="anchor" id="network_look_up"></a>
Use the [description lookup tool](#description_look_up) above to identify specific commodity or technology names. Type the name in the box below to generate a corresponding network diagram for that commodity or technology. The slider can be used to view different upstream levels of the network diagram.

In [9]:
w = widgets.Text(value='RSH')
display(w)
def f(w):
    if len(w)>0:
        
        df1 = pd.read_sql("SELECT comm_name, comm_desc FROM commodities WHERE comm_name='" + w + "'", con)
        df1['desc'] = df1['comm_desc'].str.replace('#','').str.strip()
        df1['comm_tech'] = df1['comm_name']
        df1['type'] = 'commodity'

        df2 = pd.read_sql("SELECT * FROM technologies WHERE tech='" + w + "'", con)
        df2['desc'] = df2['tech_desc'].str.replace('#','').str.strip()
        df2['comm_tech'] = df2['tech']
        df2['type'] = 'technology'

        df = pd.concat([df1[['comm_tech','type','desc']], df2[['comm_tech','type','desc']]])

    
        if len(df)>0:
            def show_desc(level):
                display(Markdown(df['desc'][0]))
                final_dem = df['comm_tech'][0]
                df_graph = return_flowd_table(final_dem,level)
                args = create_args_flowd(df_graph)
                colors, quick_run_dot_fmt = return_format_colors()
                args.update(colors)
                #o_str = 'rankdir = "LR" ;'
                #r_str = 'rankdir = "LR" ; \n\t size="12,12";'
                #quick_run_dot_fmt = quick_run_dot_fmt.replace(o_str, r_str)
                dot_graph = quick_run_dot_fmt % args
                display(graphviz.Source(dot_graph))
            w2 = widgets.IntSlider(value=1,min=0,max=10,step=1,description='Level:',disabled=False,continuous_update=True,orientation='horizontal',readout=True,readout_format='d')
            w = widgets.interactive(show_desc, level=w2)
            controls_rows(w)
        else:
            print('')

    else:
        print('')
            


out = widgets.interactive_output(f, {'w': w})
display(out)

Text(value='RSH')

Output()

###  Technology/Commodity Look-up Tool <a class="anchor" id="tech_lookup"></a>
Use the tool below to retrieve the description for any technology or commodity within the database. Type the commodity or technology name in the box below to view the description. Note that names are case sensitive.

In [10]:
w = widgets.Text(value='RSH')
display(w)
def f(w):
    df = pd.read_sql("SELECT * FROM commodities WHERE comm_name='" + w + "'", con)
    if len(df)==0:
        df = pd.read_sql("SELECT * FROM technologies WHERE tech='" + w + "'", con)

    if len(df)>0:
        try:
            display(Markdown((df['comm_desc'].values[0].replace('#', '').strip())))
        except:
            display(Markdown(df['tech_desc'].values[0].replace('#', '').strip()))
    else:
        print('')


out = widgets.interactive_output(f, {'w': w})
display(out)


Text(value='RSH')

Output()